In [1]:
# [1]
import torch
import pandas as pd
import json
import os
import logging
from sentence_transformers import SentenceTransformer

# --- Configuración del Logging ---
# Define el nombre del archivo de log basado en el nombre del notebook
log_file = 'embedding.log'

# Limpia handlers previos para evitar logs duplicados al re-ejecutar la celda
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configura el logging para que escriba en un archivo y en la consola
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

# --- Configuración de Pandas ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 1000) 

logging.info("Inicio del notebook 'embedding.ipynb'. Librerías importadas y logging configurado.")

/home/nicolas/Escritorio/Inteligencia_Artificial/Inteligencia Artificial/proyecto/Proyecto_IA_Turistica/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-07 00:33:04,828 - INFO - Inicio del notebook 'embedding.ipynb'. Librerías importadas y logging configurado.


In [3]:
# [2]
# --- Carga de Datos ---
# Se construye la ruta relativa desde la ubicación del notebook ('cuadernos/rag/') hasta el archivo.
csv_file_path = '../../datos_limpios/sitios_cali_maestro.csv'
df_sitios = None  # Inicializamos la variable para manejarla fuera del try/except

try:
    # Intenta cargar el archivo CSV en un DataFrame de Pandas
    df_sitios = pd.read_csv(csv_file_path)
    logging.info(f"Éxito: Se cargaron {len(df_sitios)} registros desde '{csv_file_path}'.")
except FileNotFoundError:
    # Manejo de error si el archivo no se encuentra en la ruta especificada
    logging.error(f"No se pudo encontrar el archivo en la ruta: '{csv_file_path}'")
    logging.error(f"Ubicación actual de ejecución: {os.getcwd()}")
    print("Error: Archivo no encontrado. Revisa la ruta en la celda y la estructura de directorios.")

# Visualización de las primeras filas para confirmar la carga
if df_sitios is not None:
    display(df_sitios.head(3))

2025-10-07 00:08:37,954 - INFO - Éxito: Se cargaron 2470 registros desde '../../datos_limpios/sitios_cali_maestro.csv'.


,place_id,nombre,direccion,tipo_territorio,departamento,pais,latitud,longitud,calificacion,total_calificaciones,comentarios,nivel_precios,telefono,tipo_telefono,sitio_web,entrada_accesible,es_reservable,estado_negocio,tags,abre_lunes,abre_martes,abre_miercoles,abre_jueves,abre_viernes,abre_sabado,abre_domingo,abierto_24h,fuente,direccion_original
0,ChIJ4ym1rp-mMI4RdMtQYqwmVGE,la casona valluna,carrera 38d numero 4c - 54 nueva granada,Ciudad,Valle del Cauca,Colombia,3.426,-76.546,4.400,3865.000,"terrible experiencia, las marranitas llegaron ...",Normal,+57 310 4472694,Móvil,http://www.lacasonavalluna.com/,True,True,operational,"['establishment', 'food', 'point_of_interest',...",True,True,True,True,True,True,True,False,Google API,"cra. 38d #4c - 54, nueva granada, cali, valle ..."
1,ChIJ0eAQM2ymMI4RINEvdgewB84,cantina la 15 granada cali norte,calle 15 norte numero 9n-62 santa monica resid...,Ciudad,Valle del Cauca,Colombia,3.460,-76.534,4.600,6021.000,un lugar donde visitas y quieres volver. muy a...,Caro,+57 300 9133447,Móvil,https://www.cantinala15.com/,True,True,operational,"['bar', 'establishment', 'food', 'point_of_int...",True,True,True,True,True,True,True,False,Google API,"cl. 15 nte. #9n-62, santa monica residential, ..."
2,ChIJK_ylEV2mMI4RA75TTO_cMZ4,restaurante los amigos,carrera 8 numero 20-41 san nicolas,Ciudad,Valle del Cauca,Colombia,3.452,-76.524,4.100,129.000,"super, comida casera con un ambiente genial. |...",Económico,+57 602 8854000,Móvil,https://www.planetacolombia.com/amigos-restaur...,NaN,NaN,operational,"['establishment', 'food', 'point_of_interest',...",True,True,True,True,True,True,True,True,Google API,"cra. 8 #20-41, san nicolas, cali, valle del ca..."


In [4]:
# [3]
# --- Limpieza de Datos ---
# Solo se ejecuta si el DataFrame se cargó correctamente en la celda anterior
if df_sitios is not None:
    # Rellenamos valores NaN (nulos) con textos por defecto para evitar errores.
    # Esto garantiza que cada campo tenga un valor antes de crear el documento semántico.
    df_sitios['tags'] = df_sitios['tags'].fillna('No especificado')
    df_sitios['nivel_precios'] = df_sitios['nivel_precios'].fillna('No especificado')
    df_sitios['comentarios'] = df_sitios['comentarios'].fillna('Sin opiniones.')
    logging.info("Preprocesamiento completado: Valores nulos en 'tags', 'nivel_precios' y 'comentarios' han sido rellenados.")
    print("Limpieza de datos nulos completada.")
else:
    logging.warning("El DataFrame 'df_sitios' no está cargado. Se omite la limpieza.")

2025-10-07 00:08:37,974 - INFO - Preprocesamiento completado: Valores nulos en 'tags', 'nivel_precios' y 'comentarios' han sido rellenados.


Limpieza de datos nulos completada.


In [5]:
# [4]
# --- Ingeniería del Documento Semántico ---

def crear_documento_semantico(sitio: dict) -> str:
    """Crea un único documento de texto a partir de un diccionario (JSON) de un sitio turístico."""
    nombre = sitio.get('nombre', 'Nombre no disponible')
    tags = sitio.get('tags', 'No especificado')
    nivel_precios = sitio.get('nivel_precios', 'No especificado')
    direccion = sitio.get('direccion', 'Ubicación no disponible')
    comentarios = sitio.get('comentarios', 'Sin opiniones.').replace('|', '. ')
    
    return (
        f"Nombre del lugar: {nombre}. Categorías: {tags}. "
        f"Nivel de precios: {nivel_precios}. Ubicación: {direccion}. "
        f"Opiniones de los visitantes: {comentarios}"
    )

if df_sitios is not None:
    # 1. Transformar el DataFrame a una lista de diccionarios (formato JSON)
    lista_sitios_json = df_sitios.to_dict(orient='records')
    logging.info(f"DataFrame transformado a una lista de {len(lista_sitios_json)} diccionarios (JSON).")

    # 2. Crear la lista de documentos semánticos (corpus) para el embedding
    corpus_documentos = [crear_documento_semantico(sitio) for sitio in lista_sitios_json]
    logging.info(f"Corpus de {len(corpus_documentos)} documentos semánticos creado.")

    # Imprimimos un ejemplo para verificar
    print("--- Ejemplo de 'Documento Semántico' ---")
    print(corpus_documentos[0])
else:
    logging.warning("El DataFrame no está cargado. Se omite la creación del documento semántico.")

2025-10-07 00:08:38,010 - INFO - DataFrame transformado a una lista de 2470 diccionarios (JSON).
2025-10-07 00:08:38,015 - INFO - Corpus de 2470 documentos semánticos creado.


--- Ejemplo de 'Documento Semántico' ---
Nombre del lugar: la casona valluna. Categorías: ['establishment', 'food', 'point_of_interest', 'restaurant']. Nivel de precios: Normal. Ubicación: carrera 38d numero 4c - 54 nueva granada. Opiniones de los visitantes: terrible experiencia, las marranitas llegaron rápido, pero todos los demás platos se demoraron más de una hora, tuvimos que preguntar varias veces por la comida, y siempre parecía que no tenían ni idea. finalmente les dijimos que nos íbamos a ir, y entonces en ese momento si entregaron la comida. la verdad si piensa ir, pues sólo marranitas, aborrajado y eso tipo de cosas que ya las tienen listas. .  hice un pedido de 100 empanadas y casi otros 100 entre marranitas y aborrajados y llegaron empacados en dos cajas, en las cuales todo venía amontonado y la mitad de los productos venían aplastados por el peso de las demás cosas, en una masa de grasa y rellenos qué no se sabía qué era qué. el ají, en vez de mandar un recipiente grande 

In [6]:
# [5]
# --- Carga del Modelo de Embedding ---
# Se carga un modelo multilingüe, ideal para proyectos escalables.
# La primera vez que se ejecute, se descargará automáticamente.
try:
    logging.info("Cargando el modelo de embedding 'paraphrase-multilingual-mpnet-base-v2'...")
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    logging.info("Modelo de embedding cargado exitosamente en memoria.")
    print("Modelo de embedding cargado exitosamente.")
except Exception as e:
    logging.critical(f"No se pudo cargar el modelo de SentenceTransformer. Error: {e}")
    print(f"Error crítico: No se pudo cargar el modelo. Verifica tu conexión a internet o la instalación de la librería.")

2025-10-07 00:08:38,022 - INFO - Cargando el modelo de embedding 'paraphrase-multilingual-mpnet-base-v2'...
2025-10-07 00:08:38,024 - INFO - Use pytorch device_name: cpu
2025-10-07 00:08:38,025 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
2025-10-07 00:08:41,787 - INFO - Modelo de embedding cargado exitosamente en memoria.


Modelo de embedding cargado exitosamente.


In [7]:
# [6]
# --- Generación de los Embeddings ---
# Esta es la parte computacionalmente más intensiva.
# 'model.encode' convierte la lista de textos en una matriz numérica (Tensor).
if 'corpus_documentos' in locals() and 'model' in locals():
    logging.info(f"Iniciando la generación de embeddings para {len(corpus_documentos)} documentos.")
    embeddings = model.encode(corpus_documentos, show_progress_bar=True, convert_to_tensor=True)
    logging.info(f"Proceso de embedding completado. Se generó una matriz de forma: {embeddings.shape}")

    print("\n--- ¡Proceso de Embedding Completado! ---")
    print(f"Forma de la matriz de embeddings: {embeddings.shape}")
    print(f"Cada uno de los {embeddings.shape[0]} sitios ahora está representado por un vector de {embeddings.shape[1]} dimensiones.")
else:
    logging.warning("No se puede generar embeddings porque el corpus o el modelo no están definidos.")

2025-10-07 00:08:41,793 - INFO - Iniciando la generación de embeddings para 2470 documentos.
Batches: 100%|██████████| 78/78 [04:00<00:00,  3.08s/it]
2025-10-07 00:12:42,448 - INFO - Proceso de embedding completado. Se generó una matriz de forma: torch.Size([2470, 768])



--- ¡Proceso de Embedding Completado! ---
Forma de la matriz de embeddings: torch.Size([2470, 768])
Cada uno de los 2470 sitios ahora está representado por un vector de 768 dimensiones.


In [8]:
# [7]
# --- Visualización de una Muestra de la Matriz de Embeddings ---
# Convertimos el tensor a un DataFrame de Pandas para una visualización más amigable.
# Esto es solo para inspección; en producción, se usaría el tensor directamente.
if 'embeddings' in locals():
    # Tomamos una muestra: las primeras 5 filas (sitios) y las primeras 10 dimensiones (columnas) del vector.
    muestra_embeddings_df = pd.DataFrame(embeddings.cpu().numpy()[:5, :10])
    
    # Asignamos nombres a las columnas para mayor claridad
    muestra_embeddings_df.columns = [f'dim_{i+1}' for i in range(10)]
    
    # Asignamos el nombre del sitio como índice para saber a qué corresponde cada vector
    muestra_embeddings_df.index = [sitio.get('nombre', 'N/A') for sitio in lista_sitios_json[:5]]
    
    print("\n--- Muestra de la Matriz de Embeddings (Primeros 5 sitios, Primeras 10 dimensiones) ---")
    display(muestra_embeddings_df)
    logging.info("Se ha mostrado una muestra de la matriz de embeddings generada.")
else:
    logging.warning("La matriz de embeddings no existe para ser visualizada.")


--- Muestra de la Matriz de Embeddings (Primeros 5 sitios, Primeras 10 dimensiones) ---


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10
la casona valluna,0.048,0.105,-0.011,-0.028,0.018,0.001,-0.010,0.031,0.010,0.127
cantina la 15 granada cali norte,-0.084,0.179,-0.012,-0.003,-0.022,-0.020,-0.064,0.004,0.012,0.176
restaurante los amigos,-0.021,0.162,-0.007,-0.066,-0.081,0.059,-0.117,0.051,0.047,0.131
storia damore granada,-0.098,0.129,-0.009,0.040,-0.037,0.020,-0.053,0.053,0.070,0.143
restaurante sazn pacfico colombiano,-0.059,0.244,-0.009,0.006,-0.014,-0.037,-0.108,-0.001,-0.085,0.152


2025-10-07 00:12:42,468 - INFO - Se ha mostrado una muestra de la matriz de embeddings generada.


In [9]:
# [8]
# --- Guardar Artefactos para su Uso en el Notebook de RAG ---

if 'embeddings' in locals() and 'lista_sitios_json' in locals():
    # 1. Definir las rutas de salida de forma relativa y segura
    output_dir = '../../datos_limpios/'
    embeddings_path = os.path.join(output_dir, 'embeddings.pt')
    json_path = os.path.join(output_dir, 'sitios_cali_maestro.json')

    # Crear el directorio si no existe
    os.makedirs(output_dir, exist_ok=True)

    # 2. Guardar el tensor de embeddings usando PyTorch
    # torch.save es la forma nativa y más eficiente de guardar tensores.
    torch.save(embeddings, embeddings_path)
    logging.info(f"Tensor de embeddings guardado exitosamente en: '{embeddings_path}'")
    print(f"Embeddings guardados en: '{embeddings_path}'")

    # 3. Guardar la lista de sitios en formato JSON (si aún no se ha hecho o para asegurar que esté actualizada)
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(lista_sitios_json, f, ensure_ascii=False, indent=4)
    logging.info(f"Datos de los sitios guardados en formato JSON en: '{json_path}'")
    print(f"Datos de los sitios guardados en: '{json_path}'")

else:
    logging.warning("No se guardaron los artefactos porque los embeddings o la lista de sitios no están definidos.")
    print("Advertencia: No se guardaron los archivos porque las variables 'embeddings' o 'lista_sitios_json' no se encontraron.")

2025-10-07 00:12:42,489 - INFO - Tensor de embeddings guardado exitosamente en: '../../datos_limpios/embeddings.pt'


2025-10-07 00:12:42,623 - INFO - Datos de los sitios guardados en formato JSON en: '../../datos_limpios/sitios_cali_maestro.json'


Embeddings guardados en: '../../datos_limpios/embeddings.pt'
Datos de los sitios guardados en: '../../datos_limpios/sitios_cali_maestro.json'
